In [16]:
%load_ext autoreload
%autoreload 2

import random
import pandas as pd
from typing import Dict, List, Tuple, Set
from problem import *
from round import *
from heuristics import *
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [8]:
def fpt(k: int, s: int):
    """
    Assumes the number of locations visited by clients is bounded by a constant
    Run k-supplier on all combination sets of locations that will be covered by facilities. Select the guess and its open facilities with the smallest objective value.
    
    PARAMETERS
    ----------
    k : int
        number of facilities to be opened
    
    RETURNS
    ----------
    facilities : List[int]
        contains facility indices that are open
    assignments : List[Tuple[int, int]]
        visited location and facility assignment indexed by each client
    """
    potential_facility_locations = list(range(s))
    
    #Remove homes from the client_location lists
    #TODO: Perhaps create mapping for the indices of people before exclusion and after?
    client_locations_excluded = []
    for person in CLIENT_LOCATIONS.values():
        new_list = [p for p in person['lid'][1:] if p in potential_facility_locations]
        if len(new_list)>0:
            client_locations_excluded.append(new_list)
    
    locations = [i for i in range(len(LOCATIONS)) if LOCATIONS[i]['lid'] < HOME_SHIFT]
    
    #Select the guess and resulting facilities that yield the smallest objective value with k-supplier
    min_obj_guess: Tuple[int, List[int], Dict[Tuple[int, int, int]:int]] = (math.inf, [], {})
    
    # TODO : allow all the locations to be facility ones
    count = 0
    for guess in powerset(list(potential_facility_locations)):
        start = time.time()
        if len(guess)==0: continue
        
        facilities = _k_supplier(list(set(guess)), locations, k)
        assignments, obj_value = assign_client_facilities(client_locations_excluded, facilities)
        
        if obj_value < min_obj_guess[0]:
            min_obj_guess = (obj_value, facilities, assignments)
        end = time.time()
        count +=1
        print(count, obj_value, end-start)
    return min_obj_guess[1], assign_facilities(min_obj_guess[1])

In [111]:
print("------CENTER OF HOMES--------")

X_home, Y_home = center_of_homes(100)
print("Recalculated Objective Value: \t" + str(calculate_objective(Y_home)))

print("------CENTER OF CENTERS------")

X_center, Y_center = center_of_centers(100)
print("Recalculated Objective Value: \t" + str(calculate_objective(Y_center)))

------CENTER OF HOMES--------
9165
HOMES 9995
LOCATIONS 9165
0.42239499925926793
[512, 0, 1, 2, 4, 517, 3, 5, 6, 521, 7, 8, 12, 525, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 26, 22, 23, 25, 27, 28, 544, 545, 29, 30, 31, 32, 33, 34, 36, 37, 42, 39, 35, 44, 45, 1583, 47, 48, 50, 49, 43, 46, 51, 55, 52, 53, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 2122, 71, 72, 73, 74, 596, 2151, 2153, 105, 2159, 24, 38, 40, 41, 219, 231, 1264, 774, 266, 276, 2859, 4913, 819, 1390, 1393, 1019]
FACILITIES 100
Recalculated Objective Value: 	0.7917222120375086
------CENTER OF CENTERS------
LOCATIONS 9165
2.7064401296229335
100
Recalculated Objective Value: 	2.3247135722810404


In [ ]:
def create_data_input():
    #Read in both the activity and residence locations
    df_activity = pd.read_csv("usa_va_charlottesville_city_activity_locations.csv").rename({"alid": "lid"}, axis = 'columns')
    df_residence = pd.read_csv("usa_va_charlottesville_city_residence_locations.csv")

    #Shift the residence lid
    df_residence['lid'] = df_residence['rlid'] + HOME_SHIFT
    locations = pd.concat([df_activity[['lid', 'longitude', 'latitude']], df_residence[['lid', 'longitude', 'latitude']]]).reset_index(drop = True)

    #Read in the client visited locations data (adults only for now)
    client_locations = pd.read_csv("usa_va_charlottesville_city_adult_activity_location_assignment_week.csv")

    #Get the coordinates of all the residential locations
    home_coords = set(df_residence[['latitude', 'longitude']].apply(tuple, axis=1).tolist())

    #Shift lids for residential areas
    client_locations['coord'] = client_locations[['latitude', 'longitude']].apply(lambda x: (x.latitude, x.longitude), axis = 1)
    client_locations.loc[client_locations.coord.isin(home_coords), 'lid'] += HOME_SHIFT
    
    #Popularity of locations, which locations are visited by which individuals
    assignments = client_locations.copy()
    assignments = assignments.groupby(['lid'])['hid'].apply(set).reset_index(name = 'pid')
    assignments = assignments.set_index('lid')
    assignments['activity'] = assignments['pid'].apply(lambda x: len(x))

    locations['activity'] = locations['lid'].apply(lambda x: assignments.at[x, 'activity'] if x in assignments.index else 0)
    locations = locations.sort_values(by = 'activity', ascending = False).reset_index(drop = True)

    client_locations = client_locations.groupby(['pid'])['lid'].apply(set).reset_index(name = 'lid')

    #Replace lid with the index of the lid in locations
    def filter(x):
        return_list = []
        for i in x:
        #Insert home locations at the front of the list
            if (i>HOME_SHIFT):
                return_list.insert(0, locations.loc[locations.lid==i].index[0])
            else:
                return_list.append(locations.loc[locations.lid==i].index[0])
        return return_list

    client_locations['lid'] = client_locations['lid'].apply(lambda x: filter(x))

    return locations.to_dict('index'), client_locations.to_dict('index')

In [13]:
LOCATIONS, CLIENT_LOCATIONS = create_data_input()

In [9]:
def _k_supplier(clients: List[int], locations: List[int], k: int):
    """
    Solves k-supplier (where client locations and facility locations may not overlap) with Hochbaum-Shmoys
    3-approximation algorithm
    
    PARAMETERS
    ----------
    distance
        diagonally-filled adjacency matrix for distances between locations
    clients
        each client is associated with a singular location
    locations
        points of interest at which facilities can be opened
    k
        number of facilities to be opened
    
    RETURNS
    ----------
    facilities : List[int]
        the facility locations that are open
    """
    l = 0
    r = 40075
    to_ret = -1
    EPSILON = 10**(-6)
    
    while r-l > EPSILON:
    
        mid = l + (r - l) / 2

        if len(_check_radius(mid, clients)) <= k:
            facilities: List[int] = _locate_facilities(mid,
                                    _check_radius(mid, clients), locations, k)
            if facilities:
                to_ret = mid
                r = mid
            else:
                l = mid
        else:
            l = mid
    
    return _locate_facilities(to_ret,_check_radius(to_ret, clients), locations, k)

def _check_radius(radius: int, clients: List[int]):
    """Determine the maximal independent set of pairiwse independent client balls with given radius
    
    PARAMETERS
    ----------
    radius
        from the binary search
    distances
        diagonally-filled adjacency matrix for distances between locations
    clients
        each client is associated with a singular location
    
    RETURNS
    ----------
    pairwise_disjoint
        maximal independent pairwise disjoint set of clients, where disjoint is defined as greater than a distance
        of 2*radius apart
    """
    
    pairwise_disjoint = set()
    
    V = set(clients)
    while len(V)!=0:
        v = V.pop()
        pairwise_disjoint.add(v)
        
        remove = set()
        for i in V:
            if calculate_distance(v, i) <= 2*radius:
                remove.add(i)
        V-=remove
    
    return pairwise_disjoint

def _locate_facilities(radius: int, pairwise_disjoint: Set[int], locations: List[int], k: int):
    """Select a facility to open within the given radius for each pairwise_disjoint client
    
    PARAMETERS
    ----------
    radius
        from the binary search
    distances
        diagonally-filled adjacency matrix for distances between locations
    pairwise_disjoint
        clients that are not within a distance of 2*radius from one another
    locations
        points of interest where facilities can be opened
    k
        number of facilities to be opened
    
    RETURNS
    ----------
    facilities: List[int]
        the locations at which facilities are opened
    """
    
    facilities = set()
    for c in pairwise_disjoint:
        for l in locations:
            if calculate_distance(c, l) <= radius:
                facilities.add(l)
                break
    
    if len(facilities) < len(pairwise_disjoint):
        return None
    
    #Check if k larger than the number of possible facility locations
    k = min(k, len(locations))
    
    #Randomly add facilities for leftover budget
    if k>len(facilities):
        unopened_facilities = set(locations)-facilities
        for i in range(k-len(facilities)):
            facilities.add(unopened_facilities.pop())
    
    return list(facilities)

In [ ]:
HOME_SHIFT=1000000000

def create_data_input():
    #Read in both the activity and residence locations
    df_activity = pd.read_csv("usa_va_charlottesville_city_activity_locations.csv").rename({"alid": "lid"}, axis = 'columns')
    df_residence = pd.read_csv("usa_va_charlottesville_city_residence_locations.csv")

    #Shift the residence lid
    df_residence['lid'] = df_residence['rlid'] + HOME_SHIFT
    locations = pd.concat([df_activity[['lid', 'longitude', 'latitude']], df_residence[['lid', 'longitude', 'latitude']]]).reset_index(drop = True)

    #Read in the client visited locations data (adults only for now)
    client_locations = pd.read_csv("usa_va_charlottesville_city_adult_activity_location_assignment_week.csv")

    #Get the coordinates of all the residential locations
    home_coords = set(df_residence[['latitude', 'longitude']].apply(tuple, axis=1).tolist())

    #Shift lids for residential areas
    client_locations['coord'] = client_locations[['latitude', 'longitude']].apply(lambda x: (x.latitude, x.longitude), axis = 1)
    client_locations.loc[client_locations.coord.isin(home_coords), 'lid'] += HOME_SHIFT
    
    #Find popularity of locations, which locations are visited by which individuals
    assignments = client_locations.copy()
    assignments = assignments.groupby(['lid'])['pid'].apply(set).reset_index(name = 'pid')
    assignments = assignments.set_index('lid')
    assignments['activity'] = assignments['pid'].apply(lambda x: len(x))

    locations['activity'] = locations['lid'].apply(lambda x: assignments.at[x, 'activity'] if x in assignments.index else 0)
    locations = locations.sort_values(by = 'activity', ascending = False).reset_index(drop = True)

    client_locations = client_locations.groupby(['pid'])['lid'].apply(set).reset_index(name = 'lid')

    #Replace lid with the index of the lid in locations
    def filter(x):
        return_list = []
        for i in x:
        #Insert home locations at the front of the list
            if (i>HOME_SHIFT):
                return_list.insert(0, locations.loc[locations.lid==i].index[0])
            else:
                return_list.append(locations.loc[locations.lid==i].index[0])
        return return_list

    client_locations['lid'] = client_locations['lid'].apply(lambda x: filter(x))

    return locations.to_dict('index'), client_locations.to_dict('index')

LOCATIONS, CLIENT_LOCATIONS = create_data_input()

In [40]:
import time

def assign_client_facilities(client_locations: List[List[int]], facilities: List[int]):
    """
    Assigns clients to their nearest facility from one of their visited locations.
    
    PARAMETERS
    ----------
        client_locations
            clients represented by index, contains a list of locations visited by each indexed client
        open_facilities
            list of facilities that are open
    RETURNS
    ----------
        assignments
            lists (visited location, facility) assignment for each client
        obj_value
    """
    if len(facilities) == 0: return []
    obj_val: int = 0
    
    assignments: List[Tuple[int, int]] = []
    
    for ind in range(len(client_locations)):
        possible_assignments = [(calculate_distance(loc, fac), loc, fac) for loc in client_locations[ind] for fac in facilities]
        
        min_loc = min(possible_assignments)
        if min_loc[0] > obj_val:
            obj_val = min_loc[0]
        assignments.append((min_loc[1], min_loc[2]))
   
    return assignments, obj_val

# TODO: change k-supplier back to list (precomputed) + put locate_facilities at bottom + try with 20
# TODO: maybe parallel (but make someone else do that)
def fpt2(k: int, s: int):
    """
    Assumes the number of locations visited by clients is bounded by a constant
    Run k-supplier on all combination sets of locations that will be covered by facilities. Select the guess and its open facilities with the smallest objective value.
    
    PARAMETERS
    ----------
    k : int
        number of facilities to be opened
    
    RETURNS
    ----------
    facilities : List[int]
        contains facility indices that are open
    assignments : List[Tuple[int, int]]
        visited location and facility assignment indexed by each client
    """
    potential_facility_locations = list(range(s))
    
    #Remove homes from the client_location lists
    #TODO: Perhaps create mapping for the indices of people before exclusion and after?
    client_locations_excluded = []
    for person in CLIENT_LOCATIONS.values():
        new_list = [p for p in person['lid'][1:] if p in potential_facility_locations]
        if len(new_list)>0:
            client_locations_excluded.append(new_list)
    
    locations = [i for i in range(len(LOCATIONS)) if LOCATIONS[i]['lid'] < HOME_SHIFT]
    
    #Select the guess and resulting facilities that yield the smallest objective value with k-supplier
    min_obj_guess: Tuple[int, List[int], Dict[Tuple[int, int, int]:int]] = (math.inf, [], {})
    
    possible_radius = sorted(calculate_possible_radius(list(potential_facility_locations), locations))
    
    counter = 0
    total_time = 0
    # TODO : allow all the locations to be facility ones
    for guess in powerset(list(potential_facility_locations)):
        start = time.time()
        if len(guess)==0: continue
        
        facilities = _k_supplier2(list(set(guess)), locations, possible_radius, k)
        assignments, obj_value = assign_client_facilities(client_locations_excluded, facilities)
        
        
        if obj_value < min_obj_guess[0]:
            min_obj_guess = (obj_value, facilities, assignments)
        
        end = time.time()
        counter+=1
        total_time+= end-start
        print(counter, obj_value, end-start, total_time/counter)
    
    return min_obj_guess[1], assign_facilities(min_obj_guess[1])

In [4]:
def calculate_possible_radius(clients: List[int], locations: List[int]):
    distances = [calculate_distance(c, l) for c in clients for l in locations] + [calculate_distance(clients[c], clients[l]) for c in range(len(clients)) for l in range(c, len(clients))]
    return distances

In [39]:
def _k_supplier2(clients: List[int], locations: List[int], possible_radius, k: int):
    """
    Solves k-supplier (where client locations and facility locations may not overlap) with Hochbaum-Shmoys
    3-approximation algorithm
    
    PARAMETERS
    ----------
    distance
        diagonally-filled adjacency matrix for distances between locations
    clients
        each client is associated with a singular location
    locations
        points of interest at which facilities can be opened
    k
        number of facilities to be opened
    
    RETURNS
    ----------
    facilities : List[int]
        the facility locations that are open
    """
    min_max = max(min([calculate_distance(c, l) for l in locations]) for c in clients)
    
    l = possible_radius.index(min_max)
    r = len(possible_radius)
    to_ret = -1
    
    while l<=r:
    
        mid = l + (r - l) // 2

        if len(_check_radius(possible_radius[mid], clients)) <= k:
            to_ret = mid
            r = mid - 1
        else:
            l = mid + 1
    
    return _locate_facilities(possible_radius[to_ret],_check_radius(possible_radius[to_ret], clients), locations, k)

def _check_radius(radius: int, clients: List[int]):
    """Determine the maximal independent set of pairiwse independent client balls with given radius
    
    PARAMETERS
    ----------
    radius
        from the binary search
    distances
        diagonally-filled adjacency matrix for distances between locations
    clients
        each client is associated with a singular location
    
    RETURNS
    ----------
    pairwise_disjoint
        maximal independent pairwise disjoint set of clients, where disjoint is defined as greater than a distance
        of 2*radius apart
    """
    
    pairwise_disjoint = set()
    
    V = set(clients)
    while len(V)!=0:
        v = V.pop()
        pairwise_disjoint.add(v)
        
        remove = set()
        for i in V:
            if calculate_distance(v, i) <= 2*radius:
                remove.add(i)
        V-=remove
    
    return pairwise_disjoint

def _locate_facilities(radius: int, pairwise_disjoint: Set[int], locations: List[int], k: int):
    """Select a facility to open within the given radius for each pairwise_disjoint client
    
    PARAMETERS
    ----------
    radius
        from the binary search
    distances
        diagonally-filled adjacency matrix for distances between locations
    pairwise_disjoint
        clients that are not within a distance of 2*radius from one another
    locations
        points of interest where facilities can be opened
    k
        number of facilities to be opened
    
    RETURNS
    ----------
    facilities: List[int]
        the locations at which facilities are opened
    """
    
    facilities = set()
    for c in pairwise_disjoint:
        for l in locations:
            if calculate_distance(c, l) <= radius:
                facilities.add(l)
                break
    
    if len(facilities) < len(pairwise_disjoint):
        return None
    
    #Check if k larger than the number of possible facility locations
    k = min(k, len(locations))
    
    #Randomly add facilities for leftover budget
    if k>len(facilities):
        #Revise to be list and checking
        unopened_facilities = set(locations)-facilities
        for i in range(k-len(facilities)):
            facilities.add(unopened_facilities.pop())
    
    return list(facilities)

[autoreload of problem failed: Traceback (most recent call last):
  File "/home/al7gc/.conda/envs/facility/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/al7gc/.conda/envs/facility/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/home/al7gc/.conda/envs/facility/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/al7gc/.conda/envs/facility/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 317, in update_class
    update_instances(old, new)
  File "/home/al7gc/.conda/envs/facility/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 276, in update_instances
    refs = gc.get_referrers(old)
KeyboardInterrupt
]


In [37]:
def _k_supplier3(clients: List[int], locations: List[int], possible_radius, k: int):
    l = 0
    r = len(possible_radius)
    to_ret = -1
    
    while l<=r:
    
        mid = l + (r - l) // 2
        if len(_check_radius(possible_radius[mid], clients)) <= k:
            facilities: List[int] = _locate_facilities(possible_radius[mid],_check_radius(possible_radius[mid], clients), locations, k)
            if facilities:
                to_ret = mid
                r = mid - 1
        else:
            l = mid + 1
    
    return _locate_facilities(possible_radius[to_ret],_check_radius(possible_radius[to_ret], clients), locations, k)

def fpt3(k: int, s: int):
    potential_facility_locations = list(range(s))
    
    #Remove homes from the client_location lists
    #TODO: Perhaps create mapping for the indices of people before exclusion and after?
    client_locations_excluded = []
    for person in CLIENT_LOCATIONS.values():
        new_list = [p for p in person['lid'][1:] if p in potential_facility_locations]
        if len(new_list)>0:
            client_locations_excluded.append(new_list)
    
    locations = [i for i in range(len(LOCATIONS)) if LOCATIONS[i]['lid'] < HOME_SHIFT]
    
    #Select the guess and resulting facilities that yield the smallest objective value with k-supplier
    min_obj_guess: Tuple[int, List[int], Dict[Tuple[int, int, int]:int]] = (math.inf, [], {})
    
    possible_radius = sorted(calculate_possible_radius(list(potential_facility_locations), locations))
    
    counter = 0
    total_time = 0
    # TODO : allow all the locations to be facility ones
    for guess in powerset(list(potential_facility_locations)):
        start = time.time()
        if len(guess)==0: continue
        
        facilities = _k_supplier3(list(set(guess)), locations, possible_radius, k)
        assignments, obj_value = assign_client_facilities(client_locations_excluded, facilities)
        
        
        if obj_value < min_obj_guess[0]:
            min_obj_guess = (obj_value, facilities, assignments)
        
        end = time.time()
        counter+=1
        total_time += (end-start)
        print(counter, obj_value, end-start, total_time/counter)
    
    return min_obj_guess[1], assign_facilities(min_obj_guess[1])

In [41]:
X_fpt, Y_fpt = fpt2(5, 25)
print(X_fpt)
print("Recalculated Objective Value: \t" + str(calculate_objective(Y_fpt)))

1 3.4593534422186054 4.097152948379517 4.097152948379517
2 3.4593534422186054 4.02099609375 4.059074521064758
3 3.4593534422186054 4.085821151733398 4.067990064620972
4 3.4593534422186054 4.048057794570923 4.0630069971084595
5 3.4593534422186054 4.070454835891724 4.064496564865112
6 1.7700538328600697 4.059037923812866 4.063586791356404
7 3.4593534422186054 4.036806583404541 4.059761047363281
8 3.4593534422186054 4.068709373474121 4.060879588127136
9 3.4593534422186054 4.080593109130859 4.063069979349772
10 1.9127481010826437 4.071002960205078 4.063863277435303
11 2.014547843127082 4.0623791217803955 4.063728354193947
12 1.7700538328603694 4.123103141784668 4.068676253159841
13 3.177877107437903 4.101034164428711 4.071165323257446
14 3.4593534422186054 4.093392848968506 4.072753003665379
15 3.4517979274767714 4.100477695465088 4.074601316452027
16 3.177877107437903 4.090383052825928 4.075587674975395
17 3.313432433675201 4.095002174377441 4.076729704351986
18 3.4593534422186054 4.10597

KeyboardInterrupt: 

In [38]:
X_fpt, Y_fpt = fpt3(5, 25)
print(X_fpt)
print("Recalculated Objective Value: \t" + str(calculate_objective(Y_fpt)))

1 3.4593534422186054 4.064459323883057 4.064459323883057
2 3.4593534422186054 4.05040168762207 4.0574305057525635
3 3.4593534422186054 4.016750812530518 4.043870608011882
4 3.4593534422186054 4.02944278717041 4.040263652801514
5 3.4593534422186054 4.0537331104278564 4.042957544326782
6 1.7700538328600697 4.020211458206177 4.039166529973348
7 3.4593534422186054 4.388705253601074 4.089100633348737
8 3.4593534422186054 4.1439573764801025 4.095957726240158
9 3.4593534422186054 4.026671409606934 4.0882592466142444
10 1.9127481010826437 4.025054454803467 4.081938767433167
11 2.014547843127082 4.031642913818359 4.077366417104548
12 1.7700538328603694 4.049738645553589 4.075064102808635
13 3.177877107437903 4.0179736614227295 4.070672530394334
14 3.4593534422186054 4.048417568206787 4.0690828902380805
15 3.4517979274767714 4.036535024642944 4.066913032531739
16 3.177877107437903 4.026362180709839 4.06437860429287
17 3.313432433675201 4.014898300170898 4.061467998168048
18 3.4593534422186054 4.

KeyboardInterrupt: 

In [35]:
X_fpt, Y_fpt = fpt(5, 25)
print(X_fpt)
print("Recalculated Objective Value: \t" + str(calculate_objective(Y_fpt)))

1 3.4593534422186054 4.015867710113525
2 3.4593534422186054 4.047598600387573
3 3.4593534422186054 4.035743236541748
4 3.4593534422186054 3.9947738647460938
5 3.4593534422186054 4.0237953662872314
6 1.7700538328600697 4.0489466190338135
7 3.4593534422186054 4.04280161857605
8 3.4593534422186054 4.009607315063477
9 3.4593534422186054 4.04019570350647
10 1.9127481010826437 4.029352426528931
11 2.014547843127082 4.032610177993774
12 1.7700538328603694 3.9936957359313965
13 3.177877107437903 4.013477563858032
14 3.4593534422186054 4.043768405914307


KeyboardInterrupt: 

In [21]:
def precompute_distances(client_locations: List[List[int]], locations: List[int]):
    G = []
    clients = set(l for loc in client_locations for l in loc)
    for l in locations:
        G.append([0 for i in len(locations)])
        for c in clients:
            if c<l:
                G[-1][c] = calculate_distance(c, l)
    return G

In [46]:
X_ind, Y_ind = independent_LP(5)
print(X_ind)
print("Recalculated Objective Value: \t" + str(calculate_objective(Y_ind)))

[autoreload of problem failed: Traceback (most recent call last):
  File "/home/al7gc/.conda/envs/facility/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/al7gc/.conda/envs/facility/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/home/al7gc/.conda/envs/facility/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/al7gc/.conda/envs/facility/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 317, in update_class
    update_instances(old, new)
  File "/home/al7gc/.conda/envs/facility/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 276, in update_instances
    refs = gc.get_referrers(old)
KeyboardInterrupt
]


5 17871
Number of variables = 112586
distances calculated 0.0038645267486572266
2.1386098861694336
Number of constraints = 243032
Objective value = 0.0


KeyboardInterrupt: 

## FPT VERSIONS ##

In [76]:
def calculate_possible_radius(clients: List[int], locations: List[int]):
    distances = [calculate_distance(c, l) for c in clients for l in locations] + [calculate_distance(clients[c], clients[l]) for c in range(len(clients)) for l in range(c, len(clients))]
    return distances

def _k_supplier(clients: List[int], locations: List[int], k: int):
    """
    Solves k-supplier (where client locations and facility locations may not overlap) with Hochbaum-Shmoys
    3-approximation algorithm
    
    PARAMETERS
    ----------
    distance
        diagonally-filled adjacency matrix for distances between locations
    clients
        each client is associated with a singular location
    locations
        points of interest at which facilities can be opened
    k
        number of facilities to be opened
    
    RETURNS
    ----------
    facilities : List[int]
        the facility locations that are open
    """
    l = 0
    r = 40075
    to_ret = -1
    EPSILON = 10**(-6)
    
    while r-l > EPSILON:
    
        mid = l + (r - l) / 2

        if len(_check_radius(mid, clients)) <= k:
            facilities: List[int] = _locate_facilities(mid,
                                    _check_radius(mid, clients), locations, k)
            if facilities:
                to_ret = mid
                r = mid
            else:
                l = mid
        else:
            l = mid
    
    return _locate_facilities(to_ret,_check_radius(to_ret, clients), locations, k)

def _check_radius(radius: int, clients: List[int]):
    """Determine the maximal independent set of pairiwse independent client balls with given radius
    
    PARAMETERS
    ----------
    radius
        from the binary search
    distances
        diagonally-filled adjacency matrix for distances between locations
    clients
        each client is associated with a singular location
    
    RETURNS
    ----------
    pairwise_disjoint
        maximal independent pairwise disjoint set of clients, where disjoint is defined as greater than a distance
        of 2*radius apart
    """
    
    pairwise_disjoint = set()
    
    V = set(clients)
    while len(V)!=0:
        v = V.pop()
        pairwise_disjoint.add(v)
        
        remove = set()
        for i in V:
            if calculate_distance(v, i) <= 2*radius:
                remove.add(i)
        V-=remove
    
    return pairwise_disjoint

def _locate_facilities(radius: int, pairwise_disjoint: Set[int], locations: List[int], k: int):
    """Select a facility to open within the given radius for each pairwise_disjoint client
    
    PARAMETERS
    ----------
    radius
        from the binary search
    distances
        diagonally-filled adjacency matrix for distances between locations
    pairwise_disjoint
        clients that are not within a distance of 2*radius from one another
    locations
        points of interest where facilities can be opened
    k
        number of facilities to be opened
    
    RETURNS
    ----------
    facilities: List[int]
        the locations at which facilities are opened
    """
    
    facilities = set()
    for c in pairwise_disjoint:
        for l in locations:
            if calculate_distance(c, l) <= radius:
                facilities.add(l)
                break
    
    if len(facilities) < len(pairwise_disjoint):
        return None
    
    #Check if k larger than the number of possible facility locations
    k = min(k, len(locations))
    
    #Randomly add facilities for leftover budget
    if k>len(facilities):
        unopened_facilities = set(locations)-facilities
        for i in range(k-len(facilities)):
            facilities.add(unopened_facilities.pop())
    
    return list(facilities)

def _k_supplier2(clients: List[int], locations: List[int], possible_radius, k: int):
    """
    Solves k-supplier (where client locations and facility locations may not overlap) with Hochbaum-Shmoys
    3-approximation algorithm
    
    PARAMETERS
    ----------
    distance
        diagonally-filled adjacency matrix for distances between locations
    clients
        each client is associated with a singular location
    locations
        points of interest at which facilities can be opened
    k
        number of facilities to be opened
    
    RETURNS
    ----------
    facilities : List[int]
        the facility locations that are open
    """
    min_max = max(min([calculate_distance(c, l) for l in locations]) for c in clients)
    
    l = possible_radius.index(min_max)
    r = len(possible_radius)
    to_ret = -1
    
    while l<=r:
    
        mid = l + (r - l) // 2

        if len(_check_radius(possible_radius[mid], clients)) <= k:
            to_ret = mid
            r = mid - 1
        else:
            l = mid + 1
    
    return _locate_facilities(possible_radius[to_ret],_check_radius(possible_radius[to_ret], clients), locations, k)

def _k_supplier3(clients: List[int], locations: List[int], possible_radius, k: int):
    l = 0
    r = len(possible_radius)
    to_ret = -1
    
    while l<=r:
    
        mid = l + (r - l) // 2
        if len(_check_radius(possible_radius[mid], clients)) <= k:
            facilities: List[int] = _locate_facilities(possible_radius[mid],_check_radius(possible_radius[mid], clients), locations, k)
            if facilities:
                to_ret = mid
                r = mid - 1
        else:
            l = mid + 1
    
    return _locate_facilities(possible_radius[to_ret],_check_radius(possible_radius[to_ret], clients), locations, k)

def precompute_distances(client_locations: List[List[int]], locations: List[int]):
    G = []
    loc_map = {}
    c_loc_map = {}
    
    clients = set(l for loc in client_locations for l in loc)
    for l_ind, l in enumerate(locations):
        loc_map[l] = l_ind
        G.append([0 for i in range(len(clients))])
        
        for c_ind, c in enumerate(clients):
            c_loc_map[c] = c_ind
            
            if c_ind < l_ind:
                G[-1][c_ind] = calculate_distance(c, l)
    
    return G, loc_map, c_loc_map

In [81]:
def fpt(k: int, s: int):
    """
    Assumes the number of locations visited by clients is bounded by a constant
    Run k-supplier on all combination sets of locations that will be covered by facilities. Select the guess and its open facilities with the smallest objective value.
    
    PARAMETERS
    ----------
    k : int
        number of facilities to be opened
    
    RETURNS
    ----------
    facilities : List[int]
        contains facility indices that are open
    assignments : List[Tuple[int, int]]
        visited location and facility assignment indexed by each client
    """
    potential_facility_locations = list(range(s))
    
    #Remove homes from the client_location lists
    #TODO: Perhaps create mapping for the indices of people before exclusion and after?
    client_locations_excluded = []
    for person in CLIENT_LOCATIONS.values():
        new_list = [p for p in person['lid'][1:] if p in potential_facility_locations]
        if len(new_list)>0:
            client_locations_excluded.append(new_list)
    
    locations = [i for i in range(len(LOCATIONS)) if LOCATIONS[i]['lid'] < HOME_SHIFT]
    
    #Select the guess and resulting facilities that yield the smallest objective value with k-supplier
    min_obj_guess: Tuple[int, List[int], Dict[Tuple[int, int, int]:int]] = (math.inf, [], {})
    
    G, loc_map, c_loc_map = precompute_distances(client_locations_excluded, locations)
    # TODO : allow all the locations to be facility ones
    count = 0
    for guess in powerset(list(potential_facility_locations)):
        start = time.time()
        if len(guess)==0: continue
        
        facilities = _k_supplier(list(guess), locations, k)
        mid = time.time()
        
        #print(mid-start)
        obj_value = assign_client_facilities(G, loc_map, c_loc_map, client_locations_excluded, facilities)
        #mid_end = time.time()
        #print(mid_end - mid)
        
        if obj_value < min_obj_guess[0]:
            min_obj_guess = (obj_value, facilities)
        end = time.time()
        count +=1
        print(count, obj_value, end-start)
    return min_obj_guess[1], assign_facilities(min_obj_guess[1])

In [56]:
def fpt2(k: int, s: int):
    """
    Assumes the number of locations visited by clients is bounded by a constant
    Run k-supplier on all combination sets of locations that will be covered by facilities. Select the guess and its open facilities with the smallest objective value.
    
    PARAMETERS
    ----------
    k : int
        number of facilities to be opened
    
    RETURNS
    ----------
    facilities : List[int]
        contains facility indices that are open
    assignments : List[Tuple[int, int]]
        visited location and facility assignment indexed by each client
    """
    potential_facility_locations = list(range(s))
    
    #Remove homes from the client_location lists
    #TODO: Perhaps create mapping for the indices of people before exclusion and after?
    client_locations_excluded = []
    for person in CLIENT_LOCATIONS.values():
        new_list = [p for p in person['lid'][1:] if p in potential_facility_locations]
        if len(new_list)>0:
            client_locations_excluded.append(new_list)
    
    locations = [i for i in range(len(LOCATIONS)) if LOCATIONS[i]['lid'] < HOME_SHIFT]
    
    #Select the guess and resulting facilities that yield the smallest objective value with k-supplier
    min_obj_guess: Tuple[int, List[int]] = (math.inf, [], {})
    
    possible_radius = sorted(calculate_possible_radius(list(potential_facility_locations), locations))
    
    counter = 0
    total_time = 0
    # TODO : allow all the locations to be facility ones
    for guess in powerset(list(potential_facility_locations)):
        start = time.time()
        if len(guess)==0: continue
        
        facilities = _k_supplier2(list(set(guess)), locations, possible_radius, k)
        obj_value = assign_client_facilities(client_locations_excluded, facilities)
        
        
        if obj_value < min_obj_guess[0]:
            min_obj_guess = (obj_value, facilities)
        
        end = time.time()
        counter+=1
        total_time+= end-start
        print(counter, obj_value, end-start, total_time/counter)
    
    return min_obj_guess[1], assign_facilities(min_obj_guess[1])

In [ ]:
def fpt3(k: int, s: int):
    potential_facility_locations = list(range(s))
    
    #Remove homes from the client_location lists
    #TODO: Perhaps create mapping for the indices of people before exclusion and after?
    client_locations_excluded = []
    for person in CLIENT_LOCATIONS.values():
        new_list = [p for p in person['lid'][1:] if p in potential_facility_locations]
        if len(new_list)>0:
            client_locations_excluded.append(new_list)
    
    locations = [i for i in range(len(LOCATIONS)) if LOCATIONS[i]['lid'] < HOME_SHIFT]
    
    #Select the guess and resulting facilities that yield the smallest objective value with k-supplier
    min_obj_guess: Tuple[int, List[int], Dict[Tuple[int, int, int]:int]] = (math.inf, [], {})
    
    possible_radius = sorted(calculate_possible_radius(list(potential_facility_locations), locations))
    
    counter = 0
    total_time = 0
    # TODO : allow all the locations to be facility ones
    for guess in powerset(list(potential_facility_locations)):
        start = time.time()
        if len(guess)==0: continue
        
        facilities = _k_supplier3(list(set(guess)), locations, possible_radius, k)
        assignments, obj_value = assign_client_facilities(client_locations_excluded, facilities)
        
        
        if obj_value < min_obj_guess[0]:
            min_obj_guess = (obj_value, facilities, assignments)
        
        end = time.time()
        counter+=1
        total_time += (end-start)
        print(counter, obj_value, end-start, total_time/counter)
    
    return min_obj_guess[1], assign_facilities(min_obj_guess[1])

In [79]:
def assign_client_facilities(G, loc_map, c_loc_map, client_locations: List[List[int]], facilities: List[int]):
    """
    Assigns clients to their nearest facility from one of their visited locations.
    
    PARAMETERS
    ----------
        client_locations
            clients represented by index, contains a list of locations visited by each indexed client
        open_facilities
            list of facilities that are open
    RETURNS
    ----------
        assignments
            lists (visited location, facility) assignment for each client
        obj_value
    """
    if len(facilities) == 0: return []
    obj_val: int = 0
    
    for ind in range(len(client_locations)):
        possible_assignments = [cost(G, loc_map[fac], c_loc_map[loc]) for loc in client_locations[ind] for fac in facilities]
        
        min_loc = min(possible_assignments)
        if min_loc > obj_val:
            obj_val = min_loc
   
    return obj_val

In [ ]:
X, Y = fpt(5, 25)
print(len(X))